<a href="https://colab.research.google.com/github/r12habh/Fake-News-Detection-Using-Machine-Learning-Techniques/blob/main/Fake_News_Detection_Using_Machine_Learning_Techniques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing requirements & Imports, Connecting with Drive

In [ ]:
import gc
gc.collect()


111

In [ ]:
!pip install keras-tuner
!pip install scikit-learn
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 1.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 30.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
import numpy as np
import pandas as pd
import os
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, GRU, Conv1D, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from keras_tuner import RandomSearch

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
!unzip /content/gdrive/MyDrive/True.csv.zip
!unzip /content/gdrive/MyDrive/Fake.csv.zip

Archive:  /content/gdrive/MyDrive/True.csv.zip
  inflating: True.csv                
Archive:  /content/gdrive/MyDrive/Fake.csv.zip
  inflating: Fake.csv                


In [ ]:

!unzip /content/gdrive/MyDrive/AllTheNews.zip

Archive:  /content/gdrive/MyDrive/AllTheNews.zip
  inflating: articles1.csv           
  inflating: articles2.csv           
  inflating: articles3.csv           


In [ ]:
!unzip /content/gdrive/MyDrive/archive.zip

Archive:  /content/gdrive/MyDrive/archive.zip
  inflating: fake.csv                


In [ ]:
!unzip /content/gdrive/MyDrive/liar_dataset.zip

Archive:  /content/gdrive/MyDrive/liar_dataset.zip
  inflating: README                  
  inflating: test.tsv                
  inflating: train.tsv               
  inflating: valid.tsv               


In [ ]:
!unzip /content/gdrive/MyDrive/FakeNewsNet-master.zip

Archive:  /content/gdrive/MyDrive/FakeNewsNet-master.zip
654361e1c8d5baa751baf1dac5032df621652280
   creating: FakeNewsNet-master/
  inflating: FakeNewsNet-master/README.md  
   creating: FakeNewsNet-master/code/
  inflating: FakeNewsNet-master/code/config.json  
  inflating: FakeNewsNet-master/code/main.py  
  inflating: FakeNewsNet-master/code/news_content_collection.py  
   creating: FakeNewsNet-master/code/resource_server/
  inflating: FakeNewsNet-master/code/resource_server/ResourceAllocator.py  
 extracting: FakeNewsNet-master/code/resource_server/__init__.py  
  inflating: FakeNewsNet-master/code/resource_server/app.py  
   creating: FakeNewsNet-master/code/resources/
  inflating: FakeNewsNet-master/code/resources/tweet_keys_file.json  
  inflating: FakeNewsNet-master/code/retweet_collection.py  
  inflating: FakeNewsNet-master/code/tweet_collection.py  
  inflating: FakeNewsNet-master/code/user_profile_collection.py  
   creating: FakeNewsNet-master/code/util/
  inflating: Fake

# Load and preprocess datasets


In [ ]:
# https://www.kaggle.com/datasets/clmentbisaillon/fake-and-real-news-dataset
# Load the initial datasets
true_data = pd.read_csv('/content/True.csv')
fake_data = pd.read_csv('/content/Fake.csv')

# Assign labels and combine datasets
true_data['label'] = 1
fake_data['label'] = 0
data = pd.concat([true_data, fake_data], ignore_index=True)
from dateutil.parser import parse
from dateutil.parser import ParserError

def parse_date(date_string):
    try:
        return parse(date_string)
    except (ParserError, TypeError):
        return None

data['date'] = data['date'].apply(parse_date)

data

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,2017-12-31,1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,2017-12-29,1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,2017-12-31,1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,2017-12-30,1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,2017-12-29,1
...,...,...,...,...,...
44893,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,2016-01-16,0
44894,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,2016-01-16,0
44895,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,2016-01-15,0
44896,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,2016-01-14,0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44898 entries, 0 to 44897
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   title    44898 non-null  object        
 1   text     44898 non-null  object        
 2   subject  44898 non-null  object        
 3   date     44888 non-null  datetime64[ns]
 4   label    44898 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 1.7+ MB


In [ ]:
# https://www.kaggle.com/datasets/snapcrack/all-the-news
# Load additional datasets
all_news_1 = pd.read_csv('/content/articles1.csv')
all_news_2 = pd.read_csv('/content/articles2.csv')
all_news_3 = pd.read_csv('/content/articles3.csv')

# Combine the additional datasets
all_news = pd.concat([all_news_1, all_news_2, all_news_3], ignore_index=True)
all_news

,Unnamed: 0,id,title,publication,author,date,year,month,url,content
0,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...
1,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood..."
2,2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017.0,4.0,NaN,"Death may be the great equalizer, but it isn’t..."
4,4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0,NaN,"SEOUL, South Korea — North Korea’s leader, ..."
...,...,...,...,...,...,...,...,...,...,...
142565,146028,218078,An eavesdropping Uber driver saved his 16-year...,Washington Post,Avi Selk,2016-12-30,2016.0,12.0,https://web.archive.org/web/20161231004909/htt...,Uber driver Keith Avila picked up a p...
142566,146029,218079,Plane carrying six people returning from a Cav...,Washington Post,Sarah Larimer,2016-12-30,2016.0,12.0,https://web.archive.org/web/20161231004909/htt...,Crews on Friday continued to search L...
142567,146030,218080,After helping a fraction of homeowners expecte...,Washington Post,Renae Merle,2016-12-30,2016.0,12.0,https://web.archive.org/web/20161231004909/htt...,When the Obama administration announced a...
142568,146031,218081,"Yes, this is real: Michigan just banned bannin...",Washington Post,Chelsea Harvey,2016-12-30,2016.0,12.0,https://web.archive.org/web/20161231004909/htt...,This story has been updated. A new law in...


In [ ]:
print(all_news.loc[3, 'title'])


Among Deaths in 2016, a Heavy Toll in Pop Music - The New York Times


In [ ]:
all_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142570 entries, 0 to 142569
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Unnamed: 0   142570 non-null  int64  
 1   id           142570 non-null  int64  
 2   title        142568 non-null  object 
 3   publication  142570 non-null  object 
 4   author       126694 non-null  object 
 5   date         139929 non-null  object 
 6   year         139929 non-null  float64
 7   month        139929 non-null  float64
 8   url          85559 non-null   object 
 9   content      142570 non-null  object 
dtypes: float64(2), int64(2), object(6)
memory usage: 10.9+ MB


In [ ]:
# Drop unwanted columns
all_news = all_news.drop(columns=['Unnamed: 0', 'id', 'year', 'month'])

# Change 'date' column to datetime dtype
all_news['date'] = pd.to_datetime(all_news['date'])

all_news['label'] = 1  # Assuming all news articles in this dataset are legitimate

# Rename 'content' column in all_news to 'text'
all_news.rename(columns={'content': 'text'}, inplace=True)

# Combine with the existing dataset
data = pd.concat([data, all_news], ignore_index=True)
data

,title,text,subject,date,label,publication,author,url
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,2017-12-31,1,NaN,NaN,NaN
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,2017-12-29,1,NaN,NaN,NaN
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,2017-12-31,1,NaN,NaN,NaN
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,2017-12-30,1,NaN,NaN,NaN
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,2017-12-29,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
187463,An eavesdropping Uber driver saved his 16-year...,Uber driver Keith Avila picked up a p...,NaN,2016-12-30,1,Washington Post,Avi Selk,https://web.archive.org/web/20161231004909/htt...
187464,Plane carrying six people returning from a Cav...,Crews on Friday continued to search L...,NaN,2016-12-30,1,Washington Post,Sarah Larimer,https://web.archive.org/web/20161231004909/htt...
187465,After helping a fraction of homeowners expecte...,When the Obama administration announced a...,NaN,2016-12-30,1,Washington Post,Renae Merle,https://web.archive.org/web/20161231004909/htt...
187466,"Yes, this is real: Michigan just banned bannin...",This story has been updated. A new law in...,NaN,2016-12-30,1,Washington Post,Chelsea Harvey,https://web.archive.org/web/20161231004909/htt...


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187468 entries, 0 to 187467
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   title        187466 non-null  object        
 1   text         187468 non-null  object        
 2   subject      44898 non-null   object        
 3   date         184817 non-null  datetime64[ns]
 4   label        187468 non-null  int64         
 5   publication  142570 non-null  object        
 6   author       126694 non-null  object        
 7   url          85559 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 11.4+ MB


In [ ]:
# https://www.kaggle.com/datasets/mrisdal/fake-news
# Load additional fake news dataset
fake_news_additional = pd.read_csv('/content/fake.csv')
fake_news_additional

,uuid,ord_in_thread,author,published,title,text,language,crawled,site_url,country,domain_rank,thread_title,spam_score,main_img_url,replies_count,participants_count,likes,comments,shares,type
0,6a175f46bcd24d39b3e962ad0f29936721db70db,0,Barracuda Brigade,2016-10-26T21:41:00.000+03:00,Muslims BUSTED: They Stole Millions In Gov’t B...,Print They should pay all the back all the mon...,english,2016-10-27T01:49:27.168+03:00,100percentfedup.com,US,25689.0,Muslims BUSTED: They Stole Millions In Gov’t B...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
1,2bdc29d12605ef9cf3f09f9875040a7113be5d5b,0,reasoning with facts,2016-10-29T08:47:11.259+03:00,Re: Why Did Attorney General Loretta Lynch Ple...,Why Did Attorney General Loretta Lynch Plead T...,english,2016-10-29T08:47:11.259+03:00,100percentfedup.com,US,25689.0,Re: Why Did Attorney General Loretta Lynch Ple...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
2,c70e149fdd53de5e61c29281100b9de0ed268bc3,0,Barracuda Brigade,2016-10-31T01:41:49.479+02:00,BREAKING: Weiner Cooperating With FBI On Hilla...,Red State : \nFox News Sunday reported this mo...,english,2016-10-31T01:41:49.479+02:00,100percentfedup.com,US,25689.0,BREAKING: Weiner Cooperating With FBI On Hilla...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
3,7cf7c15731ac2a116dd7f629bd57ea468ed70284,0,Fed Up,2016-11-01T05:22:00.000+02:00,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,Email Kayla Mueller was a prisoner and torture...,english,2016-11-01T15:46:26.304+02:00,100percentfedup.com,US,25689.0,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,0.068,http://100percentfedup.com/wp-content/uploads/...,0,0,0,0,0,bias
4,0206b54719c7e241ffe0ad4315b808290dbe6c0f,0,Fed Up,2016-11-01T21:56:00.000+02:00,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,english,2016-11-01T23:59:42.266+02:00,100percentfedup.com,US,25689.0,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,0.865,http://100percentfedup.com/wp-content/uploads/...,0,0,0,0,0,bias
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12994,f1b5d0e44803f48732bde854a9fdf95837219b12,2,replaceme,2016-10-26T23:58:00.000+03:00,NaN,It DOES allow you to put a dog face on top of ...,english,2016-10-27T00:37:46.194+03:00,zerohedge.com,US,2435.0,"Snapchat To Raise Up To $4 Billion In IPO, Val...",0.000,NaN,40,32,0,0,0,bs
12995,36011ceba3647e1bea78299b68b6fb705a1fc1ad,3,Freedumb,2016-10-27T00:02:00.000+03:00,NaN,Wait till you see what happens to the valuatio...,english,2016-10-27T00:37:46.220+03:00,zerohedge.com,US,2435.0,"Snapchat To Raise Up To $4 Billion In IPO, Val...",0.000,NaN,40,32,0,0,0,bs
12996,6995d1aa9ac99926106489b14b5530e85358059a,4,major major maj...,2016-10-27T00:06:00.000+03:00,NaN,I'm waiting for the one that puts a pussy on m...,english,2016-10-27T00:37:46.244+03:00,zerohedge.com,US,2435.0,"Snapchat To Raise Up To $4 Billion In IPO, Val...",0.000,NaN,40,32,0,0,0,bs
12997,7de8ae90eee164eb756db6c8a3772288e11d7a94,5,beemasters,2016-10-27T00:09:00.000+03:00,NaN,$4 Billion even after they are known to be kee...,english,2016-10-27T00:37:46.247+03:00,zerohedge.com,US,2435.0,"Snapchat To Raise Up To $4 Billion In IPO, Val...",0.000,NaN,40,32,0,0,0,bs


In [ ]:
fake_news_additional.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12999 entries, 0 to 12998
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   uuid                12999 non-null  object 
 1   ord_in_thread       12999 non-null  int64  
 2   author              10575 non-null  object 
 3   published           12999 non-null  object 
 4   title               12319 non-null  object 
 5   text                12953 non-null  object 
 6   language            12999 non-null  object 
 7   crawled             12999 non-null  object 
 8   site_url            12999 non-null  object 
 9   country             12823 non-null  object 
 10  domain_rank         8776 non-null   float64
 11  thread_title        12987 non-null  object 
 12  spam_score          12999 non-null  float64
 13  main_img_url        9356 non-null   object 
 14  replies_count       12999 non-null  int64  
 15  participants_count  12999 non-null  int64  
 16  like

In [ ]:
# Keep the 'author', 'site_url', 'published', 'title', and 'text' columns
columns_to_keep = ['author', 'site_url', 'published', 'title', 'text']
fake_news_additional_cleaned = fake_news_additional[columns_to_keep]

# Rename 'published' column to 'date' and 'site_url' column to 'url'
fake_news_additional_cleaned = fake_news_additional_cleaned.rename(columns={'published': 'date', 'site_url': 'url'})

# Add 'label' column and set it to 0 (assuming this dataset contains fake news)
fake_news_additional_cleaned['label'] = 0


In [ ]:
from pytz import utc

# Convert string dates to datetime objects with timezone information
fake_news_additional_cleaned['date'] = pd.to_datetime(fake_news_additional_cleaned['date'], format='%Y-%m-%dT%H:%M:%S.%f%z', errors='coerce')

# Convert timezone-aware datetime objects to UTC timezone
fake_news_additional_cleaned['date'] = fake_news_additional_cleaned['date'].apply(lambda x: x.astimezone(utc) if hasattr(x, 'tzinfo') and x.tzinfo else x).dt.tz_localize(None)

# Drop the time component and convert to string dtype
fake_news_additional_cleaned['date'] = pd.to_datetime(fake_news_additional_cleaned['date']).dt.date


In [ ]:
fake_news_additional_cleaned['date'] = pd.to_datetime(fake_news_additional_cleaned['date'])
fake_news_additional_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12999 entries, 0 to 12998
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   author  10575 non-null  object        
 1   url     12999 non-null  object        
 2   date    12999 non-null  datetime64[ns]
 3   title   12319 non-null  object        
 4   text    12953 non-null  object        
 5   label   12999 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 609.5+ KB


In [ ]:
# Merge with the existing 'data' DataFrame
data = pd.concat([data, fake_news_additional_cleaned], ignore_index=True)
data


,title,text,subject,date,label,publication,author,url
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,2017-12-31,1,NaN,NaN,NaN
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,2017-12-29,1,NaN,NaN,NaN
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,2017-12-31,1,NaN,NaN,NaN
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,2017-12-30,1,NaN,NaN,NaN
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,2017-12-29,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
200462,NaN,It DOES allow you to put a dog face on top of ...,NaN,2016-10-26,0,NaN,replaceme,zerohedge.com
200463,NaN,Wait till you see what happens to the valuatio...,NaN,2016-10-26,0,NaN,Freedumb,zerohedge.com
200464,NaN,I'm waiting for the one that puts a pussy on m...,NaN,2016-10-26,0,NaN,major major maj...,zerohedge.com
200465,NaN,$4 Billion even after they are known to be kee...,NaN,2016-10-26,0,NaN,beemasters,zerohedge.com


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200467 entries, 0 to 200466
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   title        199785 non-null  object        
 1   text         200421 non-null  object        
 2   subject      44898 non-null   object        
 3   date         197816 non-null  datetime64[ns]
 4   label        200467 non-null  int64         
 5   publication  142570 non-null  object        
 6   author       137269 non-null  object        
 7   url          98558 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 12.2+ MB


In [ ]:
# https://www.cs.ucsb.edu/~william/data/liar_dataset.zip
# Load and preprocess LIAR dataset
liar_train = pd.read_csv('/content/train.tsv', sep='\t', header=None)
liar_test = pd.read_csv('/content/test.tsv', sep='\t', header=None)
liar_valid = pd.read_csv('/content/valid.tsv', sep='\t', header=None)

# Combine LIAR train, test, and valid datasets
liar_data = pd.concat([liar_train, liar_test, liar_valid], ignore_index=True)
liar_data


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12786,3419.json,half-true,"For the first time in more than a decade, impo...","energy,oil-spill,trade",barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,a press conference
12787,12548.json,mostly-true,Says Donald Trump has bankrupted his companies...,candidates-biography,hillary-clinton,Presidential candidate,New York,democrat,40.0,29.0,69.0,76.0,7.0,a speech on the economy
12788,401.json,true,"John McCain and George Bush have ""absolutely n...",health-care,campaign-defend-america,NaN,"Washington, D.C.",none,0.0,1.0,0.0,2.0,0.0,a television ad
12789,1055.json,false,A new poll shows 62 percent support the presid...,health-care,americans-united-change,NaN,NaN,none,1.0,4.0,4.0,1.0,0.0,an Internet ad.


In [ ]:
# Assign column names
liar_data.columns = ['id', 'label', 'text', 'subject', 'speaker', 'job_title', 'state', 'party', 'barely_true', 'false', 'half_true', 'mostly_true', 'pants_on_fire', 'context']

# Rename the columns to match the existing data
liar_data_cleaned = liar_data.rename(columns={'context': 'url', 'speaker': 'author', 'job_title': 'publication'})

# Note that the Liar dataset does not have a 'title' or 'date' column
# We can create empty columns for these with None values
liar_data_cleaned['title'] = None
liar_data_cleaned['date'] = None

# Select the columns to keep
columns_to_keep = ['title', 'text', 'subject', 'date', 'label', 'publication', 'author', 'url']
liar_data_cleaned = liar_data_cleaned[columns_to_keep]

# Convert label to binary
liar_data_cleaned['label'] = liar_data_cleaned['label'].map({'true': 1, 'mostly-true': 1, 'half-true': 1, 'barely-true': 0, 'false': 0, 'pants-fire': 0})

# Merge with the existing 'data' DataFrame
data = pd.concat([data, liar_data_cleaned], ignore_index=True)
data


,title,text,subject,date,label,publication,author,url
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,2017-12-31 00:00:00,1,NaN,NaN,NaN
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,2017-12-29 00:00:00,1,NaN,NaN,NaN
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,2017-12-31 00:00:00,1,NaN,NaN,NaN
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,2017-12-30 00:00:00,1,NaN,NaN,NaN
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,2017-12-29 00:00:00,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
213253,None,"For the first time in more than a decade, impo...","energy,oil-spill,trade",None,1,President,barack-obama,a press conference
213254,None,Says Donald Trump has bankrupted his companies...,candidates-biography,None,1,Presidential candidate,hillary-clinton,a speech on the economy
213255,None,"John McCain and George Bush have ""absolutely n...",health-care,None,1,NaN,campaign-defend-america,a television ad
213256,None,A new poll shows 62 percent support the presid...,health-care,None,0,NaN,americans-united-change,an Internet ad.


In [ ]:
data['date'] = pd.to_datetime(data['date'])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213258 entries, 0 to 213257
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   title        199785 non-null  object        
 1   text         213212 non-null  object        
 2   subject      57687 non-null   object        
 3   date         197816 non-null  datetime64[ns]
 4   label        213258 non-null  int64         
 5   publication  151794 non-null  object        
 6   author       150058 non-null  object        
 7   url          111218 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 13.0+ MB


In [ ]:
# https://github.com/KaiDMML/FakeNewsNet
# Load FakeNewsNet datasets
gossipcop_fake = pd.read_csv('/content/FakeNewsNet-master/dataset/gossipcop_fake.csv')
gossipcop_real = pd.read_csv('/content/FakeNewsNet-master/dataset/gossipcop_real.csv')
politifact_fake = pd.read_csv('/content/FakeNewsNet-master/dataset/politifact_fake.csv')
politifact_real = pd.read_csv('/content/FakeNewsNet-master/dataset/politifact_real.csv')

# Rename 'news_url' column to 'url'
gossipcop_fake = gossipcop_fake.rename(columns={'news_url': 'url'})
gossipcop_real = gossipcop_real.rename(columns={'news_url': 'url'})
politifact_fake = politifact_fake.rename(columns={'news_url': 'url'})
politifact_real = politifact_real.rename(columns={'news_url': 'url'})

# Assign labels to FakeNewsNet datasets
gossipcop_fake['label'] = 0
gossipcop_real['label'] = 1
politifact_fake['label'] = 0
politifact_real['label'] = 1

# Select relevant columns in FakeNewsNet datasets
gossipcop_fake = gossipcop_fake[['title', 'label', 'url']]
gossipcop_real = gossipcop_real[['title', 'label', 'url']]
politifact_fake = politifact_fake[['title', 'label', 'url']]
politifact_real = politifact_real[['title', 'label', 'url']]

# Combine all datasets into a single dataframe
all_datasets = [data, gossipcop_fake, gossipcop_real, politifact_fake, politifact_real]
data = pd.concat(all_datasets, ignore_index=True)
data

,title,text,subject,date,label,publication,author,url
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,2017-12-31,1,NaN,NaN,NaN
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,2017-12-29,1,NaN,NaN,NaN
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,2017-12-31,1,NaN,NaN,NaN
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,2017-12-30,1,NaN,NaN,NaN
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,2017-12-29,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
236449,Flake: “Religious tests should have no place i...,NaN,NaN,NaT,1,NaN,NaN,https://www.flake.senate.gov/public/index.cfm/...
236450,Change We Can Believe In,NaN,NaN,NaT,1,NaN,NaN,https://web.archive.org/web/20080131000131/htt...
236451,deputy director of national health statistics ...,NaN,NaN,NaT,1,NaN,NaN,http://www.youtube.com/watch?v=4O8CxZ1OD58
236452,Romneys ProLife Conversion Myth or Reality Jun...,NaN,NaN,NaT,1,NaN,NaN,http://www.youtube.com/watch?v=EhyMplwY6HY


In [ ]:
gossipcop_fake

,title,label,url
0,Did Miley Cyrus and Liam Hemsworth secretly ge...,0,www.dailymail.co.uk/tvshowbiz/article-5874213/...
1,Paris Jackson & Cara Delevingne Enjoy Night Ou...,0,hollywoodlife.com/2018/05/05/paris-jackson-car...
2,Celebrities Join Tax March in Protest of Donal...,0,variety.com/2017/biz/news/tax-march-donald-tru...
3,Cindy Crawford's daughter Kaia Gerber wears a ...,0,www.dailymail.co.uk/femail/article-3499192/Do-...
4,Full List of 2018 Oscar Nominations – Variety,0,variety.com/2018/film/news/list-2018-oscar-nom...
...,...,...,...
5318,September 11: Celebrities Remember 9/11 (TWEETS),0,www.huffingtonpost.com/2012/09/11/september-11...
5319,NASCAR owners threaten to fire drivers who pro...,0,www.dailymail.co.uk/news/article-4915674/NASCA...
5320,The 7 signs that David Beckham is definitely h...,0,www.telegraph.co.uk/men/the-filter/7-signs-dav...
5321,Ryan Gosling and Eva Mendes Did Not Get Marrie...,0,www.vanityfair.com/style/2016/09/ryan-gosling-...


In [ ]:
# Shuffle the combined data
data = data.sample(frac=1).reset_index(drop=True)
labels = data.label
data

,title,text,subject,date,label,publication,author,url
0,Kim Kardashian Gets Hysterical Over Lost Earri...,NaN,NaN,NaT,0,NaN,NaN,www.eonline.com/shows/kardashians/videos/24806...
1,None,"Barack Obama ""extended health care for wounded...",military,NaT,1,President,barack-obama,a television ad
2,Perfect Peyton Manning witness emerges in sex ...,"The haze over Feb. 29, 1996, may never be comp...",NaN,2016-03-02,1,New York Post,Mark W. Sanchez,http://nypost.com/2016/03/02/perfect-peyton-ma...
3,None,The $18.8 billion in funding for K-12 educatio...,education,NaT,1,Governor,rick-scott,a press release about his education budget pro...
4,Michelle Obama Just BLASTED Conservatives For...,"On Saturday, Michelle Obama graced Jackson Sta...",News,2016-04-24,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
236449,Teasing A Girl At A Pakistani Park Could Get Y...,Mohammed Sayed is not one of those people who ...,NaN,2016-02-23,1,NPR,Philip Reeves,http://www.npr.org/sections/goatsandsoda/2016/...
236450,South Africa's Zuma congratulates Zimbabwe's M...,JOHANNESBURG (Reuters) - South African Preside...,worldnews,2017-11-24,1,NaN,NaN,NaN
236451,Qatar retools after boycott by top auto parts ...,DOHA (Reuters) - In a dusty industrial zone in...,worldnews,2017-11-09,1,NaN,NaN,NaN
236452,Report: Kim Kardashian to Do Nude Cookbook,NaN,NaN,NaT,0,NaN,NaN,www.aceshowbiz.com/news/view/00106556.html


In [ ]:
data['text'] = data['text'].fillna('')  # Fill missing values with an empty string
data['title'] = data['title'].fillna('')  # Fill missing values with an empty string


In [ ]:
from textblob import TextBlob

# Add sentiment analysis as a feature
def get_sentiment(text):
    sentiment = TextBlob(text).sentiment.polarity
    return sentiment

data['sentiment'] = data['text'].apply(get_sentiment)
data['title_sentiment'] = data['title'].apply(get_sentiment)
data


In [ ]:
# Add sentiment analysis as a feature
def get_sentiment(text):
    sentiment = TextBlob(text).sentiment.polarity
    return sentiment

data['title_sentiment'] = data['title'].apply(get_sentiment)
data


,title,text,subject,date,label,publication,author,url,sentiment,title_sentiment
0,Kim Kardashian Gets Hysterical Over Lost Earri...,,NaN,NaT,0,NaN,NaN,www.eonline.com/shows/kardashians/videos/24806...,0.000000,-1.0
1,,"Barack Obama ""extended health care for wounded...",military,NaT,1,President,barack-obama,a television ad,0.000000,0.0
2,Perfect Peyton Manning witness emerges in sex ...,"The haze over Feb. 29, 1996, may never be comp...",NaN,2016-03-02,1,New York Post,Mark W. Sanchez,http://nypost.com/2016/03/02/perfect-peyton-ma...,0.084907,1.0
3,,The $18.8 billion in funding for K-12 educatio...,education,NaT,1,Governor,rick-scott,a press release about his education budget pro...,0.000000,0.0
4,Michelle Obama Just BLASTED Conservatives For...,"On Saturday, Michelle Obama graced Jackson Sta...",News,2016-04-24,0,NaN,NaN,NaN,0.041993,-0.6
...,...,...,...,...,...,...,...,...,...,...
236449,Teasing A Girl At A Pakistani Park Could Get Y...,Mohammed Sayed is not one of those people who ...,NaN,2016-02-23,1,NPR,Philip Reeves,http://www.npr.org/sections/goatsandsoda/2016/...,0.049807,0.0
236450,South Africa's Zuma congratulates Zimbabwe's M...,JOHANNESBURG (Reuters) - South African Preside...,worldnews,2017-11-24,1,NaN,NaN,NaN,0.108712,0.0
236451,Qatar retools after boycott by top auto parts ...,DOHA (Reuters) - In a dusty industrial zone in...,worldnews,2017-11-09,1,NaN,NaN,NaN,0.015124,0.5
236452,Report: Kim Kardashian to Do Nude Cookbook,,NaN,NaT,0,NaN,NaN,www.aceshowbiz.com/news/view/00106556.html,0.000000,0.0


In [ ]:
data[(data['text'].isna() & data['title'].isna())]

,title,text,subject,date,label,publication,author,url,sentiment,title_sentiment


In [ ]:
data

,title,text,subject,date,label,publication,author,url,sentiment,title_sentiment
0,Kim Kardashian Gets Hysterical Over Lost Earri...,,NaN,NaT,0,NaN,NaN,www.eonline.com/shows/kardashians/videos/24806...,0.000000,-1.0
1,,"Barack Obama ""extended health care for wounded...",military,NaT,1,President,barack-obama,a television ad,0.000000,0.0
2,Perfect Peyton Manning witness emerges in sex ...,"The haze over Feb. 29, 1996, may never be comp...",NaN,2016-03-02,1,New York Post,Mark W. Sanchez,http://nypost.com/2016/03/02/perfect-peyton-ma...,0.084907,1.0
3,,The $18.8 billion in funding for K-12 educatio...,education,NaT,1,Governor,rick-scott,a press release about his education budget pro...,0.000000,0.0
4,Michelle Obama Just BLASTED Conservatives For...,"On Saturday, Michelle Obama graced Jackson Sta...",News,2016-04-24,0,NaN,NaN,NaN,0.041993,-0.6
...,...,...,...,...,...,...,...,...,...,...
236449,Teasing A Girl At A Pakistani Park Could Get Y...,Mohammed Sayed is not one of those people who ...,NaN,2016-02-23,1,NPR,Philip Reeves,http://www.npr.org/sections/goatsandsoda/2016/...,0.049807,0.0
236450,South Africa's Zuma congratulates Zimbabwe's M...,JOHANNESBURG (Reuters) - South African Preside...,worldnews,2017-11-24,1,NaN,NaN,NaN,0.108712,0.0
236451,Qatar retools after boycott by top auto parts ...,DOHA (Reuters) - In a dusty industrial zone in...,worldnews,2017-11-09,1,NaN,NaN,NaN,0.015124,0.5
236452,Report: Kim Kardashian to Do Nude Cookbook,,NaN,NaT,0,NaN,NaN,www.aceshowbiz.com/news/view/00106556.html,0.000000,0.0


In [ ]:
# Preprocess the data using NLTK and regular expressions
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# Download NLTK stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

# Define a text preprocessing function
def preprocess_text(text):
    text = str(text)
    text = text.lower()
    text = re.sub(r'\W', ' ', text)  # Remove non-word characters
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = ' '.join([stemmer.stem(word) for word in text.split() if word not in stop_words])  # Stemming and stopword removal
    return text

# Apply the preprocessing function to the 'text' column
data['text'] = data['text'].apply(preprocess_text)

# Tokenize the text and split the dataset into training and testing sets
max_words = 10000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data['text'])
sequences = tokenizer.texts_to_sequences(data['text'])
word_index = tokenizer.word_index

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(sequences, data['label'], test_size=0.2, random_state=42)

# Pad the sequences to have equal length
max_sequence_length = 500
x_train = pad_sequences(x_train, maxlen=max_sequence_length)
x_test = pad_sequences(x_test, maxlen=max_sequence_length)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
x_train

array([[   0,    0,    0, ..., 1103,   34, 1883],
       [   0,    0,    0, ..., 1438,  435,  872],
       [   0,    0,    0, ..., 1949, 1467, 1535],
       ...,
       [   0,    0,    0, ..., 3270,  949, 1555],
       [   0,    0,    0, ...,    0,    0,    0],
       [   0,    0,    0, ..., 5267,  575,  605]], dtype=int32)

In [ ]:
# Calculate the size of the dataset in megabytes
total_size_in_bytes = data.memory_usage(index=True, deep=True).sum()
total_size_in_megabytes = total_size_in_bytes / (1024 * 1024)
print("Size of the whole dataset in megabytes (MB):", total_size_in_megabytes)


Size of the whole dataset in megabytes (MB): 568.5186910629272


In [ ]:
# Save the data DataFrame to a CSV file in Google Drive
data.to_csv('/content/gdrive/MyDrive/data.csv', index=False)

# Save the x_train and x_test arrays to numpy binary files in Google Drive
np.save('/content/gdrive/MyDrive/x_train.npy', x_train)
np.save('/content/gdrive/MyDrive/x_test.npy', x_test)


# News Classification with Pre-trained GloVe Embeddings, LSTM, and Hyperparameter Tuning using Keras Tuner


In [ ]:
# Load pre-trained word embeddings (GloVe)
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2023-05-06 16:34:43--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-05-06 16:34:43--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-05-06 16:34:44--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:
# Define embedding dimensions and create the embeddings index
embedding_dim = 100
embeddings_index = {}
with open('glove.6B.100d.txt') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

# Create the embedding matrix
embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector


In [ ]:
# Import necessary layers and functions from Keras
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Reshape


In [ ]:
# Define the model-building function for hyperparameter tuning
def build_model(hp):
    model = Sequential()
    model.add(Embedding(max_words, embedding_dim, input_length=max_sequence_length, weights=[embedding_matrix], trainable=False))
    model.add(Dropout(hp.Float('dropout_1', 0.1, 0.5, step=0.1)))

    # Add convolutional layers
    for i in range(hp.Int('num_conv_layers', 1, 3)):
        model.add(Conv1D(hp.Int(f'conv_{i+1}_filters', 32, 128, step=32),
                         hp.Int(f'conv_{i+1}_kernel_size', 3, 7, step=2),
                         activation='relu', padding='same'))
        model.add(BatchNormalization())

    model.add(GlobalMaxPooling1D())

    # Add dense layers
    for i in range(hp.Int('num_dense_layers', 1, 3)):
        model.add(Dense(hp.Int(f'dense_{i+1}_units', 64, 256, step=64), activation='relu'))
        model.add(Dropout(hp.Float(f'dropout_{i+2}', 0.1, 0.5, step=0.1)))

    # Add a Reshape layer to add the extra dimension
    model.add(Reshape((-1, 1)))

    # Add bidirectional LSTM and GRU layers
    model.add(Bidirectional(LSTM(hp.Int('lstm_units', 32, 128, step=32), return_sequences=True)))
    model.add(GRU(hp.Int('gru_units', 16, 64, step=16)))

    # Add output layer
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Set up the random search tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    directory='hyperparameter_tuning',
    project_name='news_classification'
)

# Display the tuner's search space summary
tuner.search_space_summary()

# Run the hyperparameter search
tuner.search(x_train, y_train, epochs=6, validation_data=(x_test, y_test), batch_size=128, verbose=1)


In [ ]:


tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    directory='hyperparameter_tuning',
    project_name='news_classification'
)

tuner.search_space_summary()

tuner.search(x_train, y_train, epochs=6, validation_data=(x_test, y_test), batch_size=128, verbose=1)



Trial 10 Complete [00h 04m 17s]
val_accuracy: 0.8970205783843994

Best val_accuracy So Far: 0.8975915312767029
Total elapsed time: 00h 30m 20s


In [ ]:
# Retrieve the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Train the best model with the full dataset
best_model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=128)

# Make predictions on the testing set
y_pred_raw = best_model.predict(x_test)
y_pred = [round(pred[0]) for pred in y_pred_raw]

# Calculate accuracy, confusion matrix, and classification report
score = accuracy_score(y_test, y_pred)
print(f'Accuracy: {round(score*100, 2)}%')

confusion_matrix = confusion_matrix(y_test, y_pred)
print(f'Confusion Matrix:\n{confusion_matrix}')

classification_report = classification_report(y_test, y_pred)
print(f'Classification Report:\n{classification_report}')

Epoch 1/10
1478/1478 [==============================] - 30s 15ms/step - loss: 0.2942 - accuracy: 0.8981 - val_loss: 0.2963 - val_accuracy: 0.8958
Epoch 2/10
1478/1478 [==============================] - 21s 14ms/step - loss: 0.2936 - accuracy: 0.8986 - val_loss: 0.2943 - val_accuracy: 0.8976
Epoch 3/10
1478/1478 [==============================] - 21s 14ms/step - loss: 0.2929 - accuracy: 0.8991 - val_loss: 0.2956 - val_accuracy: 0.8975
Epoch 4/10
1478/1478 [==============================] - 21s 14ms/step - loss: 0.2928 - accuracy: 0.8993 - val_loss: 0.2946 - val_accuracy: 0.8973
Epoch 5/10
1478/1478 [==============================] - 21s 14ms/step - loss: 0.2918 - accuracy: 0.8999 - val_loss: 0.2958 - val_accuracy: 0.8969
Epoch 6/10
1478/1478 [==============================] - 21s 14ms/step - loss: 0.2917 - accuracy: 0.9003 - val_loss: 0.2947 - val_accuracy: 0.8970
Epoch 7/10
1478/1478 [==============================] - 22s 15ms/step - loss: 0.2908 - accuracy: 0.9008 - val_loss: 0.2949 -

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, Dropout, Conv1D, BatchNormalization, GlobalMaxPooling1D, Dense, Reshape, Bidirectional, LSTM, GRU
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint

def build_model(hp):
    model = Sequential()
    model.add(Embedding(max_words, embedding_dim, input_length=max_sequence_length, weights=[embedding_matrix], trainable=False))
    model.add(Dropout(hp.Float('dropout_1', 0.1, 0.5, step=0.1)))

    for i in range(hp.Int('num_conv_layers', 1, 3)):
        model.add(Conv1D(hp.Int(f'conv_{i+1}_filters', 32, 128, step=32),
                         hp.Int(f'conv_{i+1}_kernel_size', 3, 7, step=2),
                         activation='relu', padding='same'))
        model.add(BatchNormalization())

    model.add(GlobalMaxPooling1D())

    for i in range(hp.Int('num_dense_layers', 1, 3)):
        model.add(Dense(hp.Int(f'dense_{i+1}_units', 64, 256, step=64), activation='relu'))
        model.add(Dropout(hp.Float(f'dropout_{i+2}', 0.1, 0.5, step=0.1)))

    # Add a Reshape layer to add the extra dimension
    model.add(Reshape((-1, 1)))

    model.add(Bidirectional(LSTM(hp.Int('lstm_units', 32, 128, step=32), return_sequences=True)))
    model.add(GRU(hp.Int('gru_units', 16, 64, step=16)))
    model.add(Dense(1, activation='sigmoid'))

    optimizer = Adam(learning_rate=hp.Float('learning_rate', 1e-4, 1e-2, sampling='log'))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model


In [ ]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='random_search_dir',
    project_name='text_classification'
)

early_stopping = EarlyStopping(monitor='val_loss', patience=3)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)
callbacks = [early_stopping, model_checkpoint]

tuner.search(x_train, y_train, epochs=6, validation_data=(x_test, y_test), batch_size=128, callbacks=callbacks, verbose=1)

Trial 5 Complete [00h 09m 16s]
val_accuracy: 0.8952443401018778

Best val_accuracy So Far: 0.8964707851409912
Total elapsed time: 01h 11m 25s


# Hyperparameter Optimization and Evaluation of Machine Learning Models using RandomizedSearchCV

In [ ]:
import pandas as pd
import numpy as np

# Read the data DataFrame from the CSV file
data = pd.read_csv('/content/gdrive/MyDrive/data.csv')

# Read the x_train and x_test arrays from the numpy binary files
x_train = np.load('/content/gdrive/MyDrive/x_train.npy')
x_test = np.load('/content/gdrive/MyDrive/x_test.npy')
# Replace missing values with empty strings
data['text'].fillna('', inplace=True)

In [ ]:
data

,title,text,subject,date,label,publication,author,url,sentiment,title_sentiment
0,Kim Kardashian Gets Hysterical Over Lost Earri...,,NaN,NaN,0,NaN,NaN,www.eonline.com/shows/kardashians/videos/24806...,0.000000,-1.0
1,NaN,barack obama extend health care wound troop ne...,military,NaN,1,President,barack-obama,a television ad,0.000000,0.0
2,Perfect Peyton Manning witness emerges in sex ...,haze feb 29 1996 may never complet clear anoth...,NaN,2016-03-02,1,New York Post,Mark W. Sanchez,http://nypost.com/2016/03/02/perfect-peyton-ma...,0.084907,1.0
3,NaN,18 8 billion fund k 12 educ fund highest flori...,education,NaN,1,Governor,rick-scott,a press release about his education budget pro...,0.000000,0.0
4,Michelle Obama Just BLASTED Conservatives For...,saturday michel obama grace jackson state univ...,News,2016-04-24,0,NaN,NaN,NaN,0.041993,-0.6
...,...,...,...,...,...,...,...,...,...,...
236449,Teasing A Girl At A Pakistani Park Could Get Y...,moham say one peopl particularli relish prospe...,NaN,2016-02-23,1,NPR,Philip Reeves,http://www.npr.org/sections/goatsandsoda/2016/...,0.049807,0.0
236450,South Africa's Zuma congratulates Zimbabwe's M...,johannesburg reuter south african presid jacob...,worldnews,2017-11-24,1,NaN,NaN,NaN,0.108712,0.0
236451,Qatar retools after boycott by top auto parts ...,doha reuter dusti industri zone south doha cit...,worldnews,2017-11-09,1,NaN,NaN,NaN,0.015124,0.5
236452,Report: Kim Kardashian to Do Nude Cookbook,,NaN,NaN,0,NaN,NaN,www.aceshowbiz.com/news/view/00106556.html,0.000000,0.0


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from scipy.sparse import hstack
from sklearn.model_selection import RandomizedSearchCV

# Function to optimize and evaluate a given model
def optimize_and_evaluate(model_name, estimator, params, X_train, y_train, X_test, y_test):
    print(f"Optimizing {model_name}...")  # Indicate the model being optimized
    # Perform randomized search for hyperparameter optimization
    randomized_search = RandomizedSearchCV(estimator=estimator, param_distributions=params, n_iter=5, scoring='accuracy', cv=3, random_state=42)
    randomized_search.fit(X_train, y_train)

    # Get the best parameters, score, and estimator from the randomized search
    best_params = randomized_search.best_params_
    best_score = randomized_search.best_score_
    best_estimator = randomized_search.best_estimator_

    # Evaluate the optimized model on the test set
    y_pred = best_estimator.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    confusion_mat = confusion_matrix(y_test, y_pred)
    class_report = classification_report(y_test, y_pred)

    # Print the results
    print(f"Best parameters: {best_params}")
    print(f"Best cross-validated accuracy: {best_score:.2f}")
    print(f"Test set accuracy: {accuracy:.2f}")
    print(f"Confusion Matrix:\n{confusion_mat}")
    print(f"Classification Report:\n{class_report}")
    print("="*50)


In [ ]:
# Use TfidfVectorizer for feature extraction
vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 2), max_features=5000)
X = vectorizer.fit_transform(data['text'])

# Combine the sparse matrix with the sentiment column
X = hstack((X, data['sentiment'].values[:, None])).tocsr()

y = data['label']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.linear_model import LogisticRegression
# Logistic Regression
lr_params = {
    'C': [0.1, 1, 10],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'saga'],
    'max_iter': [1000, 2000, 5000]
}

optimize_and_evaluate('Logistic Regression', LogisticRegression(), lr_params, X_train, y_train, X_test, y_test)


Optimizing Logistic Regression...
Best parameters: {'solver': 'saga', 'penalty': 'l1', 'max_iter': 1000, 'C': 1}
Best cross-validated accuracy: 0.90
Test set accuracy: 0.90
Confusion Matrix:
[[ 5522  3987]
 [  843 36939]]
Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.58      0.70      9509
           1       0.90      0.98      0.94     37782

    accuracy                           0.90     47291
   macro avg       0.89      0.78      0.82     47291
weighted avg       0.90      0.90      0.89     47291



In [ ]:
# Linear Support Vector Machine
lsvc_params = {
    'C': [0.1, 1, 10]
}
optimize_and_evaluate('Linear Support Vector Machine', LinearSVC(), lsvc_params, X_train, y_train, X_test, y_test)


Optimizing Linear Support Vector Machine...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 3 is smaller than n_iter=5. Running 3 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best parameters: {'C': 1}
Best cross-validated accuracy: 0.90
Test set accuracy: 0.90
Confusion Matrix:
[[ 5555  3954]
 [  917 36865]]
Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.58      0.70      9509
           1       0.90      0.98      0.94     37782

    accuracy                           0.90     47291
   macro avg       0.88      0.78      0.82     47291
weighted avg       0.89      0.90      0.89     47291



In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Random Forest
rf_params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20]
}
optimize_and_evaluate('Random Forest', RandomForestClassifier(), rf_params, X_train, y_train, X_test, y_test)


Optimizing Random Forest...
Best parameters: {'n_estimators': 100, 'max_depth': None}
Best cross-validated accuracy: 0.89
Test set accuracy: 0.90
Confusion Matrix:
[[ 5388  4121]
 [  462 37320]]
Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.57      0.70      9509
           1       0.90      0.99      0.94     37782

    accuracy                           0.90     47291
   macro avg       0.91      0.78      0.82     47291
weighted avg       0.90      0.90      0.89     47291



# Fine-tuning Pretrained Language Models for News Classification

In [ ]:
import pandas as pd
import numpy as np

# Read the data DataFrame from the CSV file
data = pd.read_csv('/content/gdrive/MyDrive/data.csv')

# Read the x_train and x_test arrays from the numpy binary files
x_train = np.load('/content/gdrive/MyDrive/x_train.npy')
x_test = np.load('/content/gdrive/MyDrive/x_test.npy')
# Replace missing values with empty strings
data['text'].fillna('', inplace=True)

In [ ]:
import gc
gc.collect()


45

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data['text'], data['label'], test_size=0.2, random_state=42)

In [ ]:
# Convert the dataset to BERT input format
def convert_data_to_examples(data, labels):
    InputExamples = []
    for i in range(len(data)):
        example = InputExample(guid=None, text_a=data.iloc[i], text_b=None, label=labels.iloc[i])
        InputExamples.append(example)
    return InputExamples


In [ ]:
# Updated `convert_examples_to_tf_dataset` function
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = []
    for e in examples:
        input_dict = tokenizer.encode_plus(e.text_a, add_special_tokens=True, max_length=max_length, return_token_type_ids=True, return_attention_mask=True, padding='max_length', truncation=True)
        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"], input_dict["token_type_ids"], input_dict["attention_mask"])
        features.append(InputFeatures(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label))

    def gen():
        for f in features:
            yield ({"input_ids": f.input_ids, "attention_mask": f.attention_mask, "token_type_ids": f.token_type_ids}, f.label)

    return tf.data.Dataset.from_generator(gen, ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64), ({"input_ids": tf.TensorShape([None]), "attention_mask": tf.TensorShape([None]), "token_type_ids": tf.TensorShape([None])}, tf.TensorShape([])))


In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures
import tensorflow as tf

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import pandas as pd

# Replace NaN values in input data
x_train = x_train.fillna("")
x_test = x_test.fillna("")


train_examples = convert_data_to_examples(x_train, y_train)
test_examples = convert_data_to_examples(x_test, y_test)

train_dataset = convert_examples_to_tf_dataset(train_examples, tokenizer)
test_dataset = convert_examples_to_tf_dataset(test_examples, tokenizer)


In [ ]:
# Freeze all layers except the last few layers
for layer in model.layers[:-4]:
    layer.trainable = False

# Unfreeze the last few layers
for layer in model.layers[-4:]:
    layer.trainable = True

# Compile and train the model with a smaller learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-6)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_dataset.shuffle(100).batch(16), epochs=10, batch_size=16, validation_data=test_dataset.shuffle(100).batch(16))


Epoch 1/10
11823/11823 [==============================] - 986s 79ms/step - loss: 0.5791 - accuracy: 0.7725 - val_loss: 0.4170 - val_accuracy: 0.8044
Epoch 2/10
11823/11823 [==============================] - 863s 73ms/step - loss: 0.4237 - accuracy: 0.8199 - val_loss: 0.3930 - val_accuracy: 0.8425
Epoch 3/10
11823/11823 [==============================] - 856s 72ms/step - loss: 0.3835 - accuracy: 0.8405 - val_loss: 0.3490 - val_accuracy: 0.8623
Epoch 4/10
11823/11823 [==============================] - 843s 71ms/step - loss: 0.3532 - accuracy: 0.8608 - val_loss: 0.3360 - val_accuracy: 0.8759
Epoch 5/10
11823/11823 [==============================] - 842s 71ms/step - loss: 0.3397 - accuracy: 0.8679 - val_loss: 0.3255 - val_accuracy: 0.8787
Epoch 6/10
11823/11823 [==============================] - 843s 71ms/step - loss: 0.3195 - accuracy: 0.8730 - val_loss: 0.2875 - val_accuracy: 0.8829
Epoch 7/10
11823/11823 [==============================] - 829s 70ms/step - loss: 0.3051 - accuracy: 0.8788

In [ ]:
# Freeze all layers except the last few layers
for layer in model.layers[:-4]:
    layer.trainable = False

# Unfreeze the last few layers
for layer in model.layers[-4:]:
    layer.trainable = True

# Compile and train the model with a smaller learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-6)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_dataset.shuffle(100).batch(16), epochs=4, batch_size=16, validation_data=test_dataset.shuffle(100).batch(16))


Epoch 1/4
11823/11823 [==============================] - 959s 77ms/step - loss: 0.4700 - accuracy: 0.8044 - val_loss: 0.3940 - val_accuracy: 0.8405
Epoch 2/4
11823/11823 [==============================] - 817s 69ms/step - loss: 0.3869 - accuracy: 0.8446 - val_loss: 0.3415 - val_accuracy: 0.8659
Epoch 3/4
11823/11823 [==============================] - 812s 69ms/step - loss: 0.3475 - accuracy: 0.8658 - val_loss: 0.3005 - val_accuracy: 0.8798
Epoch 4/4
11823/11823 [==============================] - 815s 69ms/step - loss: 0.3246 - accuracy: 0.8717 - val_loss: 0.3045 - val_accuracy: 0.8831


In [ ]:
import pandas as pd

def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = []
    for e in examples:
        input_dict = tokenizer.encode_plus(e.text_a, add_special_tokens=True, max_length=max_length, return_attention_mask=True, padding='max_length', truncation=True)
        input_ids, attention_mask = (input_dict["input_ids"], input_dict["attention_mask"])
        features.append(InputFeatures(input_ids=input_ids, attention_mask=attention_mask, label=e.label))

    def gen():
        for f in features:
            yield ({"input_ids": f.input_ids, "attention_mask": f.attention_mask}, f.label)

    return tf.data.Dataset.from_generator(gen, ({"input_ids": tf.int32, "attention_mask": tf.int32}, tf.int64), ({"input_ids": tf.TensorShape([None]), "attention_mask": tf.TensorShape([None])}, tf.TensorShape([])))


In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification, AutoTokenizer, TFAutoModelForSequenceClassification
import tensorflow as tf

MODEL_NAME = "roberta-base"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = TFAutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

train_dataset = convert_examples_to_tf_dataset(train_examples, tokenizer)
test_dataset = convert_examples_to_tf_dataset(test_examples, tokenizer)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)

model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(train_dataset.shuffle(100).batch(32), epochs=8, batch_size=32, validation_data=test_dataset.shuffle(100).batch(32))


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/8
5912/5912 [==============================] - 762s 121ms/step - loss: 0.6948 - accuracy: 0.5403 - val_loss: 0.6932 - val_accuracy: 0.7989
Epoch 2/8
5912/5912 [==============================] - 633s 107ms/step - loss: 0.6931 - accuracy: 0.7884 - val_loss: 0.6932 - val_accuracy: 0.7989
Epoch 3/8
5912/5912 [==============================] - 628s 106ms/step - loss: 0.6931 - accuracy: 0.7884 - val_loss: 0.6932 - val_accuracy: 0.7989
Epoch 4/8
 341/5912 [>.............................] - ETA: 8:53 - loss: 0.6931 - accuracy: 0.7854

KeyboardInterrupt: ignored

In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures
import tensorflow as tf

# Use the base pre-trained model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Increase the maximum length of input sequences
max_length = 256

train_examples = convert_data_to_examples(x_train, y_train)
test_examples = convert_data_to_examples(x_test, y_test)

train_dataset = convert_examples_to_tf_dataset(train_examples, tokenizer, max_length)
test_dataset = convert_examples_to_tf_dataset(test_examples, tokenizer, max_length)

# Learning rate scheduler
lr_scheduler = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=5e-5,
    decay_steps=10000,
    decay_rate=0.9)

# Compile and train the model with a smaller learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_scheduler)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

# Train for more epochs and monitor the validation loss
model.fit(train_dataset.shuffle(100).batch(16),
          epochs=16,
          batch_size=16,
          validation_data=test_dataset.shuffle(100).batch(16),
          callbacks=[early_stopping])


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/16
11823/11823 [==============================] - 1567s 128ms/step - loss: 0.6963 - accuracy: 0.7791 - val_loss: 0.6931 - val_accuracy: 0.7989
Epoch 2/16
11823/11823 [==============================] - 1400s 118ms/step - loss: 0.6932 - accuracy: 0.7927 - val_loss: 0.6931 - val_accuracy: 0.7989
Epoch 3/16
 3507/11823 [=======>......................] - ETA: 14:54 - loss: 0.6931 - accuracy: 0.7911

KeyboardInterrupt: ignored

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

unique_classes = np.unique(y_train)
class_weights = compute_class_weight('balanced', classes=unique_classes, y=y_train)
class_weights = dict(enumerate(class_weights))

# Train for more epochs, apply class weights, and monitor the validation loss
model.fit(train_dataset.shuffle(100).batch(16),
          epochs=20,
          batch_size=32,
          validation_data=test_dataset.shuffle(100).batch(16),
          callbacks=[early_stopping],
          class_weight=class_weights)


Epoch 1/20
10024/10024 [==============================] - 1194s 118ms/step - loss: 0.6931 - accuracy: 0.7722 - val_loss: 0.6931 - val_accuracy: 0.8183
Epoch 2/20
10024/10024 [==============================] - 1186s 118ms/step - loss: 0.6931 - accuracy: 0.7736 - val_loss: 0.6931 - val_accuracy: 0.8183
Epoch 3/20
10024/10024 [==============================] - 1181s 118ms/step - loss: 0.6931 - accuracy: 0.7728 - val_loss: 0.6931 - val_accuracy: 0.8183
